## Test pertinence analyse longitudinale exports sel et eau de vie pour viz principales axes 2.2 et 2.3
2 modalités temporelles étudiées : 
* trend de 1750 à 1789
* comparaison entre 1750 et 1789

Je veux pouvoir :
- pour le sel distinguer celui qui sort du bureau de Marennes / autres bureaux de la DFLR / reste de la France
- pour l'eau-de-vie distinguer celle qui sort du bureau de Charente / autres bureaux de la DFLR

Données utilisées : Toflit (je voulais tenter avec Navigo aussi au départ mais les données ne sont pas pertinentes pour 1750)
* produits regardés : product_revolutionempire = 'Sel' / 'Eaux-de-vie et liqueurs' (en réalité on pourrait affiner en enlevant quelques produits dans cette catégorie, et en ajoutant les produits 'vin et eau-de-vie' et 'vin et eau-de-vie' de la catégorie RE 'Boissons diverses et mélangées')

#### resultats : 
- on n'a que les données par direction pour 89 si j'ai bien compris on ne peut que comparer DFLR / reste France
- pas l'impression qu'on ai un pattern marquant sur les evolutions de la part de la region dans le total des exports français sur la période étudiée

## Conclusion => abandon de la tentative d'analyse diachronique pour ces produits

In [ ]:
from poitousprint import Toflit, Portic, get_pointcalls_commodity_purposes_as_toflit_product
from vega import VegaLite
import pandas as pd

toflit_client = Toflit()
portic_client = Portic()

In [ ]:
flows_marennes_export_salt = toflit_client.get_flows(
    customs_office = 'Marennes', # peut être qu'on oublie des flows qui ne sont pas associés à un bureau en particulier, mais font bien partie de la DFLR
    start_year = 1750,
    end_year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Sel')

In [ ]:
flows_dflr_export_salt = toflit_client.get_flows(
    customs_region = 'La Rochelle',
    start_year = 1750,
    end_year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Sel')

In [ ]:
flows_national_export_salt = toflit_client.get_flows(
    start_year = 1750,
    end_year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Sel')

In [ ]:
print("Nombre d'exports de sel au niveau national entre 1750 et 1789  :", len(flows_national_export_salt))
print("Parmi ces exports, ", len(flows_dflr_export_salt)," sortent de la Direction des Fermes de La Rochelle, et ", len(flows_marennes_export_salt)," du bureau des Fermes de Marennes (pertinent uniquement en 89 pour ce bureau)")

In [ ]:
years_list = [y + 1712 for y in range(120)]
years = {}
nb_no_custom_office_flow_1789 = 0

for y in years_list:
    years[str(y)] = {
        "year": y,
        "france_export":0,
        "dflr_export": 0,
        "marennes_export": 0
    }


for f in flows_national_export_salt: 
    year = str(f['year'].split('.')[0])
    value = float(f['value']) if f['value'] != '' else 0
    years[year]['france_export'] += value        
    if f['customs_region'] == 'La Rochelle':
        years[year]['dflr_export'] += value
        if f['customs_office'] == 'Marennes':
            years[year]['marennes_export'] += value
        elif f['customs_office'] == '' and year == 1789:
            nb_no_custom_office_flow_1789 += 1

part_by_year = []
for year, values in years.items():
    part_by_year.append({
        "year": year,
        "type": "tout DFLR",
        "portion": values['dflr_export'] / values['france_export'] if  values['france_export'] > 0 else 0
    })
    part_by_year.append({
        "year": year,
        "type": "marennes",
        "portion": values['marennes_export'] / values['dflr_export'] if  values['dflr_export'] > 0 else 0
    })
    """

    part_by_year.append({
        "year": year,
        "type": "reste de la France",
        "portion": 1 - values['la_rochelle_total'] / values['france_total'] if  values['france_total'] > 0 else 0
    })
    """
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution globale de la part des échanges de La Rochelle par rapport à l'ensemble de la France",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal", 
            "title": "année"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Part Marennes dans sa direction, et la direction par rapport au total des exports de la France"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Type de flux",
            "scale": {
                "range": ["orange", "blue", "white"]
            }
        }
    }
}, pd.DataFrame([y for y in part_by_year if int(y['year']) > 1717 and int(y['year']) < 1790]))

In [ ]:
print("Le nombre d'exports de sel rattachés à la DFLR, mais pas à un bureau particulier s'élève à ", nb_no_custom_office_flow_1789)

In [ ]:
flows_national_export_edv = toflit_client.get_flows(
    start_year = 1750,
    end_year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Eaux-de-vie et liqueurs')

In [ ]:
print("Nombre d'exports d'eau de vie au niveau national entre 1750 et 1789  :", len(flows_national_export_edv))

In [ ]:
years_list = [y + 1712 for y in range(120)]
years = {}
nb_no_custom_office_flow_1789 = 0
customs_offices = set()

for y in years_list:
    years[str(y)] = {
        "year": y,
        "france_export":0,
        "dflr_export": 0,
        "charente_export": 0
    }


for f in flows_national_export_edv: 
    year = str(f['year'].split('.')[0])
    value = float(f['value']) if f['value'] != '' else 0
    years[year]['france_export'] += value  
    if f['customs_region'] == 'La Rochelle':
        years[year]['dflr_export'] += value
        customs_offices.add(f['customs_office'])
        if f['customs_office'] == 'Charente':
            years[year]['charente_export'] += value
        elif f['customs_office'] == '' and year == 1789:
            nb_no_custom_office_flow_1789 += 1

part_by_year = []
for year, values in years.items():
    part_by_year.append({
        "year": year,
        "type": "tout DFLR",
        "portion": values['dflr_export'] / values['france_export'] if  values['france_export'] > 0 else 0
    })
    part_by_year.append({
        "year":year,
        "type": "charente",
        "portion": values['charente_export'] / values['dflr_export'] if  values['dflr_export'] > 0 else 0
    })
    """

    part_by_year.append({
        "year": year,
        "type": "reste de la France",
        "portion": 1 - values['la_rochelle_total'] / values['france_total'] if  values['france_total'] > 0 else 0
    })
    """
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution globale de la part des échanges de La Rochelle par rapport à l'ensemble de la France",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal", 
            "title": "année"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Part Charente dans sa direction, et la direction par rapport au total des exports de la France"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Type de flux",
            "scale": {
                "range": ["orange", "blue", "white"]
            }
        }
    }
}, pd.DataFrame([y for y in part_by_year if int(y['year']) > 1717 and int(y['year']) < 1790]))


In [ ]:
print("Le nombre d'exports de d'eau de vie rattachés à la DFLR, mais pas à un bureau particulier s'élève à ", nb_no_custom_office_flow_1789)

### Archive de mes tentatives avec Navigo (on a beaucoup plus de données pour 1789, mais rien pour 1750)

In [ ]:
pointcalls_poitou_1789_out = portic_client.get_pointcalls(
    source_subset = 'Poitou_1789',
    ## besoin de filtrer sur les amirautés pertinentes (celles conservées dans source subset 1789)
    start_year = '1750',
    end_year = '1789',
    pointcall_action = 'Out'
)

print("nombre de pointcalls out de la direction des fermes de la rochelle en 1789 :", len(pointcalls_poitou_1789_out))

In [ ]:
pointcalls_poitou_1789_out_with_revolution_empire_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls_poitou_1789_out, product_classification="product_revolutionempire")
# print(pointcalls_poitou_1789_out_with_revolution_empire_products[0:3])

In [ ]:
# filtrer les pointcalls pour lesquels la cargaison contient du sel
pointcalls_poitou_1789_out_with_salt = []
number_of_pointcalls_from_marennes = 0

for pointcall in pointcalls_poitou_1789_out_with_revolution_empire_products:
    for commodity_purpose in pointcall['commodity_purposes']:
        if (commodity_purpose['commodity_as_toflit'] == 'Sel'):
            pointcalls_poitou_1789_out_with_salt.append(pointcall)
            if pointcall['ferme_bureau'] == 'Marennes':
                number_of_pointcalls_from_marennes += 1
        
print("Nombre de pointcalls pertinents trouvés ('Out' du Poitou en 1789 et transportant du sel) :", len(pointcalls_poitou_1789_out_with_salt))
print("Parmi ces pointcalls, ", number_of_pointcalls_from_marennes," sortent du bureau des Fermes de Marennes")